In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
import os

In [ ]:

class VehicleNeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers=[128, 64, 32]):
        super(VehicleNeuralNetwork, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)


class VehicleDataProcessor:
    def __init__(self, csv_path):
        self.csv_path = csv_path
        self.scaler_input = StandardScaler()
        self.scaler_output = StandardScaler()
        self.input_columns = []
        self.output_columns = []
        
    def preprocess_dataset(self):
        """Preprocess the CSV dataset for supervised learning"""
        print("Loading dataset...")
        df = pd.read_csv(self.csv_path)
        
        # Remove rows with NaN or infinite values
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()
        
        print(f"Dataset shape after cleaning: {df.shape}")
        
        # Create shifted dataframe for k+1 timestamp
        df_next = df.shift(-1)
        
        # Remove the last row as it won't have a next timestamp
        df = df[:-1]
        df_next = df_next[:-1]
        
        # Store original column names
        original_cols = df.columns.tolist()
        
        # Rename columns for current and next timesteps
        df_current = df.copy()
        df_next = df_next.copy()
        
        # Rename columns to distinguish current and next
        df_current.columns = [col + '_curr' for col in original_cols]
        df_next.columns = [col + '_next' for col in original_cols]
        
        # Combine current and next dataframes
        df_combined = pd.concat([df_current, df_next], axis=1)
        
        # Remove timestamp columns
        timestamp_cols = [col for col in df_combined.columns if 'timestamp' in col]
        df_combined = df_combined.drop(columns=timestamp_cols)
        
        # Remove k+1 control inputs (throttle_pwm, steering_pwm, sampling_time)
        control_cols_next = ['throttle_pwm_next', 'steering_pwm_next', 'sampling_time_next']
        df_combined = df_combined.drop(columns=control_cols_next, errors='ignore')
        
        # Define input and output columns
        self.input_columns = [col for col in df_combined.columns if col.endswith('_curr')]
        self.output_columns = [col for col in df_combined.columns if col.endswith('_next')]
        
        print(f"Input features: {len(self.input_columns)}")
        print(f"Output features: {len(self.output_columns)}")
        
        return df_combined
    
    def prepare_training_data(self, df_processed, test_size=0.2):
        """Prepare training and validation datasets"""
        
        # Extract input and output data
        X = df_processed[self.input_columns].values.astype(np.float32)
        y = df_processed[self.output_columns].values.astype(np.float32)
        
        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42
        )
        
        # Normalize the data
        X_train_scaled = self.scaler_input.fit_transform(X_train)
        X_test_scaled = self.scaler_input.transform(X_test)
        
        y_train_scaled = self.scaler_output.fit_transform(y_train)
        y_test_scaled = self.scaler_output.transform(y_test)
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test_scaled, dtype=torch.float32)
        
        return (X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor)

class VehicleNNTrainer:
    def __init__(self, model, device='cpu'):
        self.model = model.to(device)
        self.device = device
        self.train_losses = []
        self.val_losses = []
        
    def train_model(self, X_train, X_val, y_train, y_val, 
                   epochs=100, batch_size=64, learning_rate=0.001):
        """Train the neural network model"""
        
        # Create data loaders
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
        
        print("Starting training...")
        
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                
                optimizer.zero_grad()
                outputs = self.model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            
            with torch.no_grad():
                for batch_x, batch_y in val_loader:
                    batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                    outputs = self.model(batch_x)
                    loss = criterion(outputs, batch_y)
                    val_loss += loss.item()
            
            # Calculate average losses
            avg_train_loss = train_loss / len(train_loader)
            avg_val_loss = val_loss / len(val_loader)
            
            self.train_losses.append(avg_train_loss)
            self.val_losses.append(avg_val_loss)
            
            # Learning rate scheduling
            scheduler.step(avg_val_loss)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}")
                print(f"Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
                print(f"Learning Rate: {optimizer.param_groups[^4_0]['lr']:.8f}")
                print("-" * 50)
    
    def plot_training_history(self, save_path='/tmp/training_history.png'):
        """Plot training and validation loss"""
        plt.figure(figsize=(10, 6))
        plt.plot(self.train_losses, label='Training Loss')
        plt.plot(self.val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training History')
        plt.legend()
        plt.grid(True)
        plt.savefig(save_path)
        plt.close()
        print(f"Training history plot saved to {save_path}")

def main():
    # Configuration
    csv_path = '/tmp/vehicle_data_log.csv'
    model_save_path = '/tmp/vehicle_nn_model.pth'
    scaler_save_path = '/tmp/vehicle_scalers.pkl'
    config_save_path = '/tmp/vehicle_model_config.pkl'
    
    # Check if CSV file exists
    if not os.path.exists(csv_path):
        print(f"CSV file not found: {csv_path}")
        print("Please run the data logger first to collect training data.")
        return
    
    # Process data
    processor = VehicleDataProcessor(csv_path)
    df_processed = processor.preprocess_dataset()
    
    # Prepare training data
    X_train, X_test, y_train, y_test = processor.prepare_training_data(df_processed)
    
    print(f"Training data shape: {X_train.shape}")
    print(f"Training labels shape: {y_train.shape}")
    
    # Create model
    input_dim = X_train.shape[^4_1]
    output_dim = y_train.shape[^4_1]
    
    model = VehicleNeuralNetwork(input_dim, output_dim)
    
    # Train model
    trainer = VehicleNNTrainer(model)
    trainer.train_model(X_train, X_test, y_train, y_test, epochs=100)
    
    # Save model
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")
    
    # Save scalers and configuration
    model_config = {
        'input_dim': input_dim,
        'output_dim': output_dim,
        'input_columns': processor.input_columns,
        'output_columns': processor.output_columns,
        'scaler_input': processor.scaler_input,
        'scaler_output': processor.scaler_output
    }
    
    with open(config_save_path, 'wb') as f:
        pickle.dump(model_config, f)
    
    print(f"Model configuration saved to {config_save_path}")
    
    # Plot training history
    trainer.plot_training_history()
    
    print("Training completed successfully!")

if __name__ == '__main__':
    main()
```


## 2. Vehicle Simulator Script

```python
#!/usr/bin/env python
# File: src/simulation/vehicle_nn_simulator.py

import rospy
import threading
import time
import numpy as np
import torch
import torch.nn as nn
from std_msgs.msg import Float64MultiArray, Float64
from sensor_msgs.msg import Imu, LaserScan
from geometry_msgs.msg import PoseStamped, Twist
import tf.transformations
import pickle
import os

class VehicleNeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers=[128, 64, 32]):
        super(VehicleNeuralNetwork, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

class VehicleNNSimulator:
    def __init__(self):
        rospy.init_node('vehicle_nn_simulator', anonymous=True)
        
        # Parameters
        self.model_path = rospy.get_param('~model_path', '/tmp/vehicle_nn_model.pth')
        self.config_path = rospy.get_param('~config_path', '/tmp/vehicle_model_config.pkl')
        self.publish_rate = rospy.get_param('~publish_rate', 20.0)  # Hz
        self.wheelbase = rospy.get_param('~wheelbase', 0.25)
        
        # Load model and configuration
        self.load_model_and_config()
        
        # State variables
        self.current_state = None
        self.current_throttle_pwm = 1500.0  # Neutral
        self.current_steering_pwm = 1500.0  # Neutral
        self.current_sampling_time = 0.05
        self.last_cmd_time = time.time()
        self.cmd_timeout = 1.0  # seconds
        
        # Thread safety
        self.state_lock = threading.Lock()
        self.cmd_lock = threading.Lock()
        
        # Publishers
        self.pose_pub = rospy.Publisher('/simulated_pose', PoseStamped, queue_size=1)
        self.imu_pub = rospy.Publisher('/simulated_imu', Imu, queue_size=1)
        self.scan_pub = rospy.Publisher('/simulated_scan', LaserScan, queue_size=1)
        self.battery_pub = rospy.Publisher('/simulated_battery', Float64MultiArray, queue_size=1)
        self.velocity_pub = rospy.Publisher('/simulated_velocity', Twist, queue_size=1)
        
        # Subscribers
        rospy.Subscriber('/cmd_pwm', Float64MultiArray, self.cmd_callback)
        
        # Initialize state
        self.initialize_state()
        
        # Start simulation threads
        self.running = True
        self.simulation_thread = threading.Thread(target=self.simulation_loop)
        self.publish_thread = threading.Thread(target=self.publish_loop)
        
        self.simulation_thread.start()
        self.publish_thread.start()
        
        rospy.loginfo("Vehicle NN Simulator initialized")
    
    def load_model_and_config(self):
        """Load the trained model and configuration"""
        try:
            # Load configuration
            with open(self.config_path, 'rb') as f:
                self.config = pickle.load(f)
            
            # Create and load model
            self.model = VehicleNeuralNetwork(
                self.config['input_dim'], 
                self.config['output_dim']
            )
            self.model.load_state_dict(torch.load(self.model_path, map_location='cpu'))
            self.model.eval()
            
            rospy.loginfo("Model and configuration loaded successfully")
            
        except Exception as e:
            rospy.logerr(f"Failed to load model: {e}")
            rospy.signal_shutdown("Model loading failed")
    
    def initialize_state(self):
        """Initialize the vehicle state"""
        # Create initial state vector based on input columns
        state_dict = {}
        
        # Initialize with reasonable default values
        for col in self.config['input_columns']:
            if 'pose_x' in col:
                state_dict[col] = 0.0
            elif 'pose_y' in col:
                state_dict[col] = 0.0
            elif 'pose_yaw' in col:
                state_dict[col] = 0.0
            elif 'vel_x' in col:
                state_dict[col] = 0.0
            elif 'vel_y' in col:
                state_dict[col] = 0.0
            elif 'yaw_rate' in col:
                state_dict[col] = 0.0
            elif 'throttle_pwm' in col:
                state_dict[col] = 1500.0
            elif 'steering_pwm' in col:
                state_dict[col] = 1500.0
            elif 'battery_voltage' in col:
                state_dict[col] = 12.0
            elif 'imu_' in col:
                state_dict[col] = 0.0
            elif 'scan_angle_' in col:
                state_dict[col] = 5.0  # Default range
            else:
                state_dict[col] = 0.0
        
        # Convert to numpy array in correct order
        self.current_state = np.array([state_dict[col] for col in self.config['input_columns']], dtype=np.float32)
        
        rospy.loginfo("Initial state initialized")
    
    def cmd_callback(self, msg):
        """Handle incoming PWM commands"""
        with self.cmd_lock:
            if len(msg.data) >= 2:
                self.current_throttle_pwm = msg.data[^4_0]
                self.current_steering_pwm = msg.data[^4_1]
                if len(msg.data) >= 3:
                    self.current_sampling_time = msg.data[^4_2]
                self.last_cmd_time = time.time()
    
    def get_current_commands(self):
        """Get current commands with timeout handling"""
        with self.cmd_lock:
            if time.time() - self.last_cmd_time > self.cmd_timeout:
                # Timeout - return neutral commands
                return 1500.0, 1500.0, self.current_sampling_time
            else:
                return self.current_throttle_pwm, self.current_steering_pwm, self.current_sampling_time
    
    def update_state_with_commands(self, throttle_pwm, steering_pwm, sampling_time):
        """Update current state with new commands"""
        with self.state_lock:
            # Update command-related state variables
            for i, col in enumerate(self.config['input_columns']):
                if 'throttle_pwm' in col:
                    self.current_state[i] = throttle_pwm
                elif 'steering_pwm' in col:
                    self.current_state[i] = steering_pwm
                elif 'sampling_time' in col:
                    self.current_state[i] = sampling_time
    
    def predict_next_state(self):
        """Predict next state using the neural network"""
        with self.state_lock:
            # Normalize input
            input_normalized = self.config['scaler_input'].transform(self.current_state.reshape(1, -1))
            input_tensor = torch.tensor(input_normalized, dtype=torch.float32)
            
            # Predict
            with torch.no_grad():
                output_normalized = self.model(input_tensor).numpy()
            
            # Denormalize output
            output = self.config['scaler_output'].inverse_transform(output_normalized)
            
            return output.flatten()
    
    def update_state_from_prediction(self, predicted_output):
        """Update current state with predicted values"""
        with self.state_lock:
            # Map predicted output back to current state
            output_dict = {}
            for i, col in enumerate(self.config['output_columns']):
                output_dict[col] = predicted_output[i]
            
            # Update current state with predicted values
            for i, col in enumerate(self.config['input_columns']):
                # Convert from '_next' to '_curr' naming
                next_col = col.replace('_curr', '_next')
                if next_col in output_dict:
                    self.current_state[i] = output_dict[next_col]
    
    def simulation_loop(self):
        """Main simulation loop"""
        rate = rospy.Rate(self.publish_rate)
        
        while not rospy.is_shutdown() and self.running:
            try:
                # Get current commands
                throttle_pwm, steering_pwm, sampling_time = self.get_current_commands()
                
                # Update state with current commands
                self.update_state_with_commands(throttle_pwm, steering_pwm, sampling_time)
                
                # Predict next state
                predicted_output = self.predict_next_state()
                
                # Update state with prediction
                self.update_state_from_prediction(predicted_output)
                
                rate.sleep()
                
            except Exception as e:
                rospy.logerr(f"Simulation error: {e}")
                rate.sleep()
    
    def get_state_value(self, key_pattern):
        """Get state value by column pattern"""
        with self.state_lock:
            for i, col in enumerate(self.config['input_columns']):
                if key_pattern in col:
                    return self.current_state[i]
            return 0.0
    
    def publish_loop(self):
        """Publishing loop for ROS messages"""
        rate = rospy.Rate(self.publish_rate)
        
        while not rospy.is_shutdown() and self.running:
            try:
                current_time = rospy.Time.now()
                
                # Publish pose
                pose_msg = PoseStamped()
                pose_msg.header.stamp = current_time
                pose_msg.header.frame_id = "map"
                
                pose_x = self.get_state_value('pose_x')
                pose_y = self.get_state_value('pose_y')
                pose_yaw = self.get_state_value('pose_yaw')
                
                pose_msg.pose.position.x = pose_x
                pose_msg.pose.position.y = pose_y
                pose_msg.pose.position.z = 0.0
                
                q = tf.transformations.quaternion_from_euler(0, 0, pose_yaw)
                pose_msg.pose.orientation.x = q[^4_0]
                pose_msg.pose.orientation.y = q[^4_1]
                pose_msg.pose.orientation.z = q[^4_2]
                pose_msg.pose.orientation.w = q[^4_3]
                
                self.pose_pub.publish(pose_msg)
                
                # Publish IMU
                imu_msg = Imu()
                imu_msg.header.stamp = current_time
                imu_msg.header.frame_id = "base_link"
                
                imu_msg.linear_acceleration.x = self.get_state_value('imu_ax')
                imu_msg.linear_acceleration.y = self.get_state_value('imu_ay')
                imu_msg.linear_acceleration.z = self.get_state_value('imu_az')
                
                imu_msg.angular_velocity.x = self.get_state_value('imu_gx')
                imu_msg.angular_velocity.y = self.get_state_value('imu_gy')
                imu_msg.angular_velocity.z = self.get_state_value('imu_gz')
                
                self.imu_pub.publish(imu_msg)
                
                # Publish velocity
                vel_msg = Twist()
                vel_msg.linear.x = self.get_state_value('vel_x')
                vel_msg.linear.y = self.get_state_value('vel_y')
                vel_msg.angular.z = self.get_state_value('yaw_rate')
                
                self.velocity_pub.publish(vel_msg)
                
                # Publish battery voltage
                battery_msg = Float64MultiArray()
                battery_msg.data = [self.get_state_value('battery_voltage')]
                self.battery_pub.publish(battery_msg)
                
                # Publish laser scan
                scan_msg = LaserScan()
                scan_msg.header.stamp = current_time
                scan_msg.header.frame_id = "base_link"
                scan_msg.angle_min = -np.pi
                scan_msg.angle_max = np.pi
                scan_msg.angle_increment = 2 * np.pi / 360
                scan_msg.range_min = 0.1
                scan_msg.range_max = 10.0
                
                ranges = []
                for i in range(360):
                    range_val = self.get_state_value(f'scan_angle_{i}')
                    ranges.append(max(range_val, 0.1))  # Ensure minimum range
                
                scan_msg.ranges = ranges
                self.scan_pub.publish(scan_msg)
                
                rate.sleep()
                
            except Exception as e:
                rospy.logerr(f"Publishing error: {e}")
                rate.sleep()
    
    def shutdown(self):
        """Shutdown the simulator"""
        self.running = False
        if hasattr(self, 'simulation_thread'):
            self.simulation_thread.join()
        if hasattr(self, 'publish_thread'):
            self.publish_thread.join()
        rospy.loginfo("Vehicle NN Simulator shutdown complete")

if __name__ == '__main__':
    try:
        simulator = VehicleNNSimulator()
        rospy.on_shutdown(simulator.shutdown)
        rospy.spin()
    except rospy.ROSInterruptException:
        pass
```


## Launch Files

```xml
<!-- File: launch/vehicle_nn_training.launch -->
<launch>
    <node name="vehicle_nn_trainer" pkg="vehicle_control" type="vehicle_nn_trainer.py" output="screen">
        <param name="csv_file" value="/tmp/vehicle_data_log.csv"/>
        <param name="model_save_path" value="/tmp/vehicle_nn_model.pth"/>
        <param name="config_save_path" value="/tmp/vehicle_model_config.pkl"/>
    </node>
</launch>
```

```xml
<!-- File: launch/vehicle_nn_simulation.launch -->
<launch>
    <node name="vehicle_nn_simulator" pkg="vehicle_control" type="vehicle_nn_simulator.py" output="screen">
        <param name="model_path" value="/tmp/vehicle_nn_model.pth"/>
        <param name="config_path" value="/tmp/vehicle_model_config.pkl"/>
        <param name="publish_rate" value="20.0"/>
        <param name="wheelbase" value="0.25"/>
    </node>
    
    <!-- Visualization -->
    <node name="rviz" pkg="rviz" type="rviz" args="-d $(find vehicle_control)/config/simulation.rviz"/>
</launch>
```


## Usage Instructions

**Training the Model:**

```bash
# First collect data using the data logger
rosrun vehicle_control vehicle_data_logger.py

# Train the neural network model
python src/learning/vehicle_nn_trainer.py
```

**Running the Simulation:**

```bash
# Launch the simulator
roslaunch vehicle_control vehicle_nn_simulation.launch

# Send PWM commands to test
rostopic pub /cmd_pwm std_msgs/Float64MultiArray "data: [1600, 1400]"
```


## Key Features

**Data Preprocessing:**

- Automatically arranges k and k+1 timestamp data in the same row
- Removes timestamps and future control inputs as specified
- Handles missing data and normalizes features
- Saves scalers and configuration for simulation use

**Neural Network:**

- Multi-layer feedforward architecture with dropout for regularization
- Configurable hidden layers and learning parameters
- Training history visualization and model checkpointing
- Proper train/validation split with early stopping

**Simulation:**

- Multi-threaded architecture for real-time performance
- Publishes all sensor data in correct ROS message formats
- Handles command timeouts gracefully (defaults to neutral)
- Thread-safe state management
- Configurable publishing rates and parameters

The system provides a complete pipeline from data collection to model training to real-time simulation, enabling you to test controllers on the learned vehicle dynamics model.

<div style="text-align: center">⁂</div>

---

# print('...initialising')

import rospy
import numpy as np
from std_msgs.msg import Float64
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()
car.throttle_gain = 0.45
throttle_limit = (-0.6, 0.6)
steering_limit = (-0.6, 0.6)
car.steering = 0
car.throttle = 0

def throttle(msg):
car.throttle = np.clip(msg.data, *throttle_limit)

def steering(msg):
car.steering = np.clip(msg.data, *steering_limit)

if __name__ == "__main__":
rospy.init_node('MOVE_CAR')
print('starting car ... DONE\nto move publish data on topics\n\t/pwm/throttle\n\tpwm/steering')
rospy.Subscriber('pwm/throttle', Float64, throttle)
rospy.Subscriber('pwm/steering', Float64, steering)
rospy.spin()

print('Execution Stopped!')
car.steering = 0
car.throttle = -0.1

modify the above code for moving the car to write an actuator script based on the above explorer controller that publishes the cmd_pwm and runs at a frequency of given sampling time

Here's the modified actuator script that interfaces with the JetRacer hardware and responds to the explorer controller's PWM commands:

```python
#!/usr/bin/env python
# File: src/actuators/jetracer_actuator.py

print('...initialising JetRacer actuator')
import rospy
import numpy as np
from std_msgs.msg import Float64MultiArray
from jetracer.nvidia_racecar import NvidiaRacecar
import time
import threading

class JetRacerActuator:
    def __init__(self):
        # Initialize the JetRacer hardware
        self.car = NvidiaRacecar()
        self.car.throttle_gain = rospy.get_param('~throttle_gain', 0.45)
        
        # Control limits (normalized values for JetRacer)
        self.throttle_limit = rospy.get_param('~throttle_limit', [-0.6, 0.6])
        self.steering_limit = rospy.get_param('~steering_limit', [-0.6, 0.6])
        
        # PWM to normalized value conversion parameters
        # PWM range typically 1000-2000, with 1500 as center
        self.pwm_center = rospy.get_param('~pwm_center', 1500)
        self.pwm_range = rospy.get_param('~pwm_range', 500)  # +/- from center
        
        # Safety parameters
        self.emergency_stop = False
        self.last_cmd_time = time.time()
        self.cmd_timeout = rospy.get_param('~cmd_timeout', 1.0)  # seconds
        self.default_sampling_time = rospy.get_param('~default_sampling_time', 0.1)
        
        # Current control values
        self.current_throttle = 0.0
        self.current_steering = 0.0
        self.current_sampling_time = self.default_sampling_time
        
        # Thread safety
        self.control_lock = threading.Lock()
        
        # Initialize car to safe state
        self.car.steering = 0
        self.car.throttle = 0
        
        # ROS setup
        rospy.init_node('jetracer_actuator', anonymous=True)
        
        # Subscribers
        rospy.Subscriber('/cmd_pwm', Float64MultiArray, self.cmd_pwm_callback)
        rospy.Subscriber('/emergency_stop', Float64MultiArray, self.emergency_stop_callback)
        
        # Publishers for feedback
        self.status_pub = rospy.Publisher('/actuator_status', Float64MultiArray, queue_size=1)
        
        # Control loop timer (will be updated based on sampling time)
        self.control_timer = None
        self.start_control_loop()
        
        print('JetRacer actuator initialized successfully!')
        print('Listening for PWM commands on /cmd_pwm topic')
        print('PWM format: [throttle_pwm, steering_pwm, sampling_time(optional)]')
        print('PWM range: 1000-2000 (1500 = neutral)')
        
    def pwm_to_normalized(self, pwm_value):
        """Convert PWM value (1000-2000) to normalized value (-1.0 to 1.0)"""
        # Clamp PWM to reasonable range
        pwm_clamped = np.clip(pwm_value, 1000, 2000)
        
        # Convert to normalized range
        normalized = (pwm_clamped - self.pwm_center) / self.pwm_range
        return np.clip(normalized, -1.0, 1.0)
    
    def cmd_pwm_callback(self, msg):
        """Handle incoming PWM commands from explorer controller"""
        if len(msg.data) < 2:
            rospy.logwarn("Invalid PWM command: need at least throttle and steering")
            return
            
        with self.control_lock:
            # Extract PWM values
            throttle_pwm = msg.data[0]
            steering_pwm = msg.data[1]
            
            # Optional sampling time
            if len(msg.data) >= 3:
                new_sampling_time = max(msg.data[2], 0.01)  # Minimum 10ms
                if abs(new_sampling_time - self.current_sampling_time) > 0.005:
                    self.current_sampling_time = new_sampling_time
                    self.restart_control_loop()
            
            # Convert PWM to normalized values
            throttle_normalized = self.pwm_to_normalized(throttle_pwm)
            steering_normalized = self.pwm_to_normalized(steering_pwm)
            
            # Apply limits
            self.current_throttle = np.clip(throttle_normalized, *self.throttle_limit)
            self.current_steering = np.clip(steering_normalized, *self.steering_limit)
            
            # Update command timestamp
            self.last_cmd_time = time.time()
            
            rospy.logdebug(f"PWM Command - Throttle: {throttle_pwm:.0f} -> {self.current_throttle:.3f}, "
                          f"Steering: {steering_pwm:.0f} -> {self.current_steering:.3f}, "
                          f"Sampling: {self.current_sampling_time:.3f}s")
    
    def emergency_stop_callback(self, msg):
        """Handle emergency stop commands"""
        if len(msg.data) > 0 and msg.data[0] > 0:
            self.emergency_stop = True
            rospy.logwarn("EMERGENCY STOP ACTIVATED!")
        else:
            self.emergency_stop = False
            rospy.loginfo("Emergency stop deactivated")
    
    def start_control_loop(self):
        """Start the control loop timer"""
        if self.control_timer is not None:
            self.control_timer.shutdown()
        
        control_frequency = 1.0 / self.current_sampling_time
        self.control_timer = rospy.Timer(
            rospy.Duration(self.current_sampling_time), 
            self.control_loop_callback
        )
        
        rospy.loginfo(f"Control loop started at {control_frequency:.1f} Hz "
                     f"(sampling time: {self.current_sampling_time:.3f}s)")
    
    def restart_control_loop(self):
        """Restart control loop with new sampling time"""
        rospy.loginfo(f"Updating sampling time to {self.current_sampling_time:.3f}s")
        self.start_control_loop()
    
    def control_loop_callback(self, event):
        """Main control loop - applies commands to hardware"""
        try:
            with self.control_lock:
                current_time = time.time()
                
                # Check for command timeout
                if current_time - self.last_cmd_time > self.cmd_timeout:
                    # Timeout - apply safe defaults
                    target_throttle = 0.0
                    target_steering = 0.0
                    if current_time - self.last_cmd_time > self.cmd_timeout + 1.0:
                        rospy.logwarn_throttle(2.0, "Command timeout - applying safe defaults")
                else:
                    target_throttle = self.current_throttle
                    target_steering = self.current_steering
                
                # Apply emergency stop
                if self.emergency_stop:
                    target_throttle = -0.1  # Light braking
                    target_steering = 0.0
                
                # Apply commands to hardware
                self.car.throttle = target_throttle
                self.car.steering = target_steering
                
                # Publish status for monitoring
                self.publish_status(target_throttle, target_steering)
                
        except Exception as e:
            rospy.logerr(f"Control loop error: {e}")
            # Safe fallback
            self.car.throttle = 0.0
            self.car.steering = 0.0
    
    def publish_status(self, throttle, steering):
        """Publish actuator status for monitoring"""
        status_msg = Float64MultiArray()
        status_msg.data = [
            throttle,
            steering,
            self.current_sampling_time,
            1.0 if self.emergency_stop else 0.0,
            time.time() - self.last_cmd_time  # Time since last command
        ]
        self.status_pub.publish(status_msg)
    
    def shutdown(self):
        """Safe shutdown procedure"""
        rospy.loginfo("Shutting down JetRacer actuator...")
        
        # Stop the car safely
        self.car.steering = 0
        self.car.throttle = -0.1  # Light braking
        time.sleep(0.5)
        self.car.throttle = 0
        
        # Stop control timer
        if self.control_timer is not None:
            self.control_timer.shutdown()
        
        print('JetRacer actuator shutdown complete!')

def main():
    try:
        # Create actuator instance
        actuator = JetRacerActuator()
        
        # Setup shutdown handler
        rospy.on_shutdown(actuator.shutdown)
        
        # Keep the node running
        rospy.loginfo("JetRacer actuator ready - waiting for PWM commands...")
        rospy.spin()
        
    except rospy.ROSInterruptException:
        rospy.loginfo("Interrupted by user")
    except Exception as e:
        rospy.logerr(f"Actuator error: {e}")
    finally:
        # Ensure car is stopped
        try:
            car = NvidiaRacecar()
            car.steering = 0
            car.throttle = 0
            print('Emergency stop applied!')
        except:
            pass

if __name__ == "__main__":
    main()
```


## Launch File for Complete System

```xml
<!-- File: launch/jetracer_exploration.launch -->
<launch>
    <!-- JetRacer Actuator -->
    <node name="jetracer_actuator" pkg="vehicle_control" type="jetracer_actuator.py" output="screen">
        <param name="throttle_gain" value="0.45"/>
        <param name="throttle_limit" value="[-0.6, 0.6]"/>
        <param name="steering_limit" value="[-0.6, 0.6]"/>
        <param name="pwm_center" value="1500"/>
        <param name="pwm_range" value="500"/>
        <param name="cmd_timeout" value="1.0"/>
        <param name="default_sampling_time" value="0.1"/>
    </node>
    
    <!-- Exploring Controller -->
    <node name="exploring_controller" pkg="vehicle_control" type="exploring_controller.py" output="screen">
        <param name="throttle_pwm_range" value="[1450, 1550]"/>
        <param name="steering_pwm_center" value="1500"/>
        <param name="steering_pwm_range" value="[1300, 1700]"/>
        <param name="sampling_time_range" value="[0.05, 0.2]"/>
        <param name="safe_distance" value="0.5"/>
        <param name="min_gap_width" value="10"/>
        <param name="exploration_interval" value="10.0"/>
    </node>
    
    <!-- Data Logger (optional) -->
    <node name="vehicle_data_logger" pkg="vehicle_control" type="vehicle_data_logger.py" output="screen">
        <param name="csv_file" value="/tmp/jetracer_exploration_data.csv"/>
        <param name="wheelbase" value="0.25"/>
        <param name="fuse_imu" value="false"/>
    </node>
</launch>
```
